In [1]:
import torch
from transformers import AutoTokenizer
import datasets
from utils_generator import GenerationModel, auroc, define_threshold

import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, random_split
torch.manual_seed(9)

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-04-07 09:27:58.728160: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-07 09:27:59.807360: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-04-07 09:27:59.807715: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libn

In [2]:
args = {
    "batch_size":16,
    "num_workers":0,
    "shuffle":True,
    "drop_last":False,
    "scenario":"s1",
    "divergence":"Renyi",
    "alpha":0.1,
    "temperature":1,
    "r":0.95
    }

In [3]:
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-de-en")

model = GenerationModel("Helsinki-NLP/opus-mt-de-en")

/home/funny_shannon/.local/lib/python3.8/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


cuda


In [4]:
data_de = datasets.load_dataset('Helsinki-NLP/tatoeba_mt','deu-eng',split=datasets.Split.TEST, 
                                verification_mode= datasets.VerificationMode.NO_CHECKS)
data_de_train, data_de_test = random_split(data_de, [0.7, 0.3])

data_fr = datasets.load_dataset('Helsinki-NLP/tatoeba_mt','eng-fra', split=datasets.Split.TEST, 
                                verification_mode= datasets.VerificationMode.NO_CHECKS)\
                                    .rename_columns({'sourceLang': 'targetlang', 'targetlang': 'sourceLang', 
                                                     'sourceString': 'targetString', 'targetString': 'sourceString'})
data_fr_train, data_fr_test = random_split(data_fr, [0.3, 0.7])

# data_nld = datasets.load_dataset('Helsinki-NLP/tatoeba_mt','eng-nld',split=datasets.Split.TEST,
#                                   verification_mode= datasets.VerificationMode.NO_CHECKS)\
#                                     .rename_columns({'sourceLang': 'targetlang', 'targetlang': 'sourceLang', 
#                                                      'sourceString': 'targetString', 'targetString': 'sourceString'})

Found cached dataset tatoeba_mt (/home/funny_shannon/.cache/huggingface/datasets/Helsinki-NLP___tatoeba_mt/deu-eng/0.0.0/01e819f3f64a772a2ca70949061d295d3a2dc99d05183fe4776a3be23f75f619)
Found cached dataset tatoeba_mt (/home/funny_shannon/.cache/huggingface/datasets/Helsinki-NLP___tatoeba_mt/eng-fra/0.0.0/01e819f3f64a772a2ca70949061d295d3a2dc99d05183fe4776a3be23f75f619)


In [5]:
data_de_test[0]

{'sourceLang': 'deu',
 'targetlang': 'eng',
 'sourceString': 'Schach ist schwer.',
 'targetString': 'Chess is difficult.'}

In [6]:
loader_de_train = DataLoader(data_de_train, batch_size=args['batch_size'], num_workers=args['num_workers'], 
                       shuffle=args['shuffle'], drop_last=args['drop_last'])

loader_de_test = DataLoader(data_de_test, batch_size=args['batch_size'], num_workers=args['num_workers'], 
                       shuffle=args['shuffle'], drop_last=args['drop_last'])

loader_fr_train = DataLoader(data_fr_train, batch_size=args['batch_size'], num_workers=args['num_workers'], 
                       shuffle=args['shuffle'], drop_last=args['drop_last'])

loader_fr_test = DataLoader(data_fr_test, batch_size=args['batch_size'], num_workers=args['num_workers'], 
                       shuffle=args['shuffle'], drop_last=args['drop_last'])

# loader_nld = DataLoader(data_nld, batch_size=args['batch_size'], num_workers=args['num_workers'], 
#                        shuffle=args['shuffle'], drop_last=args['drop_last'])

In [7]:
x_de_train = tokenizer(next(iter(loader_de_train))["sourceString"], return_tensors="pt", padding=True)
x_de_test = tokenizer(next(iter(loader_de_test))["sourceString"], return_tensors="pt", padding=True)


x_fr_train = tokenizer(next(iter(loader_fr_train))["sourceString"], return_tensors="pt", padding=True)
x_fr_test = tokenizer(next(iter(loader_fr_test))["sourceString"], return_tensors="pt", padding=True)

# x_nld = tokenizer(next(iter(loader_nld))["sourceString"], return_tensors="pt", padding=True)

In [8]:
model.generate_bag(loader_de_train)

Generate bag of distribution:   0%|          | 0/769 [00:00<?, ?it/s]/home/funny_shannon/.local/lib/python3.8/site-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 512 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Generate bag of distribution: 100%|██████████| 769/769 [02:25<00:00,  5.29it/s]


tensor([[5.6455e-02, 1.7664e-06, 2.6444e-03,  ..., 1.7109e-06, 1.6938e-06,
         0.0000e+00],
        [1.5261e-01, 1.8251e-06, 5.2010e-04,  ..., 1.6925e-06, 1.6531e-06,
         0.0000e+00],
        [1.0669e-01, 1.8658e-06, 1.7397e-03,  ..., 1.6949e-06, 1.6892e-06,
         0.0000e+00],
        ...,
        [2.1777e-01, 1.9683e-06, 7.3236e-04,  ..., 1.7877e-06, 1.7694e-06,
         0.0000e+00],
        [2.3058e-01, 2.0039e-06, 1.5595e-04,  ..., 1.8619e-06, 1.7918e-06,
         0.0000e+00],
        [1.2758e-01, 1.9173e-06, 1.9801e-03,  ..., 1.7499e-06, 1.7492e-06,
         0.0000e+00]], device='cuda:0')

In [9]:
anomaly= model.anomaly_score(x_de_test, args["divergence"], args["scenario"], args["alpha"], args["temperature"])

In [10]:
anomaly

tensor([0.1092, 0.0622, 0.1151, 0.1638, 0.0893, 0.0708, -0.0000, 0.0777, 0.0777,
        0.1567, 0.0527, 0.0929, 0.0483, 0.0884, 0.0603, 0.0612],
       device='cuda:0')

In [11]:
anomaly_fr= model.anomaly_score(x_fr_train,args["divergence"], args["scenario"], args["alpha"], args["temperature"])

In [12]:
list_anomaly_in = model.anomaly_score_loader(loader_de_test, args["divergence"],
                                   args["scenario"], args["alpha"], args["temperature"])

list_anomaly_out = model.anomaly_score_loader(loader_fr_train, args["divergence"],
                                   args["scenario"], args["alpha"], args["temperature"])

Computing anomaly score for each data: 100%|██████████| 238/238 [05:29<00:00,  1.38s/it]


In [14]:
auroc(list_anomaly_in, list_anomaly_out, True)

NameError: name 'list_anomaly_out' is not defined

In [ ]:
threshold = define_threshold(list_anomaly_out, args["r"], args["divergence"],
                                   args["scenario"], args["alpha"], args["temperature"])

In [ ]:
threshold

In [ ]:
model.classify(x_fr_test, threshold, args["divergence"], args["scenario"], args["alpha"], args["temperature"])

In [ ]:
#anomaly= model.anomaly_score(x_de_train, args["divergence"], args["scenario"], args["alpha"], args["temperature"])

In [13]:
#out = model.model.generate(**x_de, return_dict_in_generate=True,output_scores=True, num_beams=1)

In [14]:
#out.scores[0].shape

In [15]:
#torch.stack(list(out.scores), dim=0).permute(1,0,2)

In [16]:
#model.softmax(torch.stack(list(out.scores), dim=0).permute(1,0,2))

In [17]:
# anomaly_score_de, sequences_de = model.anomaly_score(x_de, 'Renyi', 0.1, temperature=1)

# anomaly_score_fr, sequences_fr = model.anomaly_score(x_fr, 'Renyi', 0.1, temperature=1)

# anomaly_score_nld, sequences_nld = model.anomaly_score(x_nld, 'Renyi', 0.1, temperature=1)

In [18]:
# anomaly_score_de

In [19]:
# anomaly_score_fr

In [20]:
# anomaly_score_nld

In [21]:
# print(torch.mean(anomaly_score_de))
# print(torch.std(anomaly_score_de))


In [22]:
# print(torch.mean(anomaly_score_fr))
# print(torch.std(anomaly_score_fr))

In [23]:
# print(torch.mean(anomaly_score_nld))
# print(torch.std(anomaly_score_nld))


In [24]:
# tokenizer.batch_decode(generated_ids.sequences, skip_special_tokens=True)

In [25]:
# x = tokenizer(data_nld[0]["targetString"], return_tensors="pt")

# generated_ids = model.generate(**x)
# tokenizer.batch_decode(generated_ids, skip_special_tokens=True)